In [1]:
from langchain.llms import OpenAI

In [30]:
import os

os.environ["OPEN_API_KEY"] = ""

In [2]:
llm = OpenAI(temperature=0.5, api_key="")

/home/purav/Desktop/python/langchain_end2end/venv/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [4]:
text = "what is the capital of gujarat"

print(llm.predict(text))

/home/purav/Desktop/python/langchain_end2end/venv/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `predict` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




The capital of Gujarat is Gandhinagar.


Prompt Templates

In [7]:
from langchain.prompts import PromptTemplate


prompt_template = PromptTemplate(input_variables=['country'],
    template = "Tell me the capital of this {country}")

prompt_template.format(country="India")

'Tell me the capital of this India'

In [10]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt_template)
chain.run("India")

/home/purav/Desktop/python/langchain_end2end/venv/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'\n\nThe capital of India is New Delhi.'

### Combining Multiple Chains Using Simple Sequential Chain

In [12]:
capital_template = PromptTemplate(input_variables=["country"],
    template = "Pleasr tell me the capital ofthe {country}")

capital_chain = LLMChain(llm=llm, prompt=capital_template)

famous_template = PromptTemplate(input_variables=["capital"],
    template = "Suggest me some amzing places to visit in {capital}")

famous_chain = LLMChain(llm=llm, prompt=famous_template)



In [16]:
from langchain.chains import SimpleSequentialChain

chain = SimpleSequentialChain(chains=[capital_chain, famous_chain])
chain.run("India")

"\n1. Red Fort - a historic fort and UNESCO World Heritage Site\n2. Qutub Minar - a 73-meter tall minaret and UNESCO World Heritage Site\n3. India Gate - a war memorial and popular picnic spot\n4. Humayun's Tomb - a stunning Mughal-era mausoleum\n5. Lotus Temple - a Bahá'í House of Worship known for its unique lotus-shaped architecture\n6. Akshardham Temple - a magnificent Hindu temple complex with beautiful architecture and gardens\n7. Jama Masjid - one of the largest mosques in India\n8. Chandni Chowk - a bustling market known for its food, shopping, and cultural experiences\n9. Rashtrapati Bhavan - the official residence of the President of India\n10. National Museum - a must-visit for history and art enthusiasts."

### Sequential Chain

In [20]:
capital_template = PromptTemplate(input_variables=["country"],
    template = "Pleasr tell me the capital ofthe {country}")

capital_chain = LLMChain(llm=llm, prompt=capital_template, output_key="capital")

famous_template = PromptTemplate(input_variables=["capital"],
    template = "Suggest me some amzing places to visit in {capital}")

famous_chain = LLMChain(llm=llm, prompt=famous_template, output_key="places")



In [21]:
from langchain.chains import SequentialChain

chain = SequentialChain(chains=[capital_chain, famous_chain],
                        input_variables=["country"],
                        output_variables=["capital", "places"])

chain({"country": "India"})

/home/purav/Desktop/python/langchain_end2end/venv/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


{'country': 'India',
 'capital': '\n\nThe capital of India is New Delhi.',
 'places': " Some amazing places to visit in New Delhi are:\n\n1. Red Fort - A UNESCO World Heritage Site, this majestic fort was built in the 17th century and is a symbol of Mughal architecture.\n\n2. Qutub Minar - Another UNESCO World Heritage Site, this 73-meter tall tower is the tallest brick minaret in the world and is surrounded by beautiful gardens.\n\n3. India Gate - A war memorial dedicated to the soldiers who died in World War I, this iconic landmark is a must-visit in New Delhi.\n\n4. Lotus Temple - This stunning temple, shaped like a lotus flower, is a Bahá'í House of Worship and welcomes people of all religions.\n\n5. Humayun's Tomb - This beautiful mausoleum was built in the 16th century and is the resting place of Mughal Emperor Humayun.\n\n6. Jama Masjid - One of the largest mosques in India, this magnificent structure was built by Mughal Emperor Shah Jahan.\n\n7. Chandni Chowk - One of the oldes

### Chatmodels with ChatOpenAI

In [31]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage,SystemMessage,AIMessage

chatllm = ChatOpenAI(api_key=os.environ["OPEN_API_KEY"], temperature=0.5, model="gpt-3.5-turbo")

In [32]:
chatllm([
    SystemMessage(content="You are comedian AI assistant"),
    HumanMessage(content="provide some comedy punchlines on AI")
])

AIMessage(content='1. "Why did the AI break up with its robot girlfriend? She couldn\'t handle its emotional bandwidth!"\n2. "I asked my AI assistant to tell me a joke, and it replied, \'I can\'t, I\'m programmed for serious business only!\'"\n3. "AI and humans have one thing in common - we both struggle to understand each other\'s emotions!"\n4. "I told my AI to make me a sandwich, and it responded, \'I\'m sorry, I\'m not programmed for food delivery. But I can order you one online!\'"\n5. "AI and procrastination have a lot in common - they both love to take forever to finish a task!"\n6. "I asked my AI to help me find love, and it replied, \'I\'m sorry, I can\'t help with matters of the heart. But I can recommend some dating apps!\'"\n7. "AI\'s favorite pick-up line: \'Are you a computer keyboard? Because you\'re just my type!\'"\n8. "Why did the AI go to therapy? It had too many unresolved bugs in its emotional software!"\n9. "AI\'s version of a dad joke: \'Why was the robot so bad 

### Prompt Template + LLM + Output Parsers

In [33]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import BaseOutputParser

In [40]:
class CommaSeperatedOutput(BaseOutputParser):
    def parse(self, text:str):
        return text.strip().split(",")

In [35]:
template = "you are a helpful assistant. when the use given any input you should generate 5 sysnonym words in a comma seperated list"
human_template = "{text}"
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template)
])

In [41]:
chain = chat_prompt|chatllm|CommaSeperatedOutput()

In [45]:
chain.invoke({"text": "rigorous"})

['strict', ' thorough', ' precise', ' exacting', ' demanding']